In [ ]:
import os                                     # operating system
import numpy as np                            # linear algebra
from PIL import Image                         # Python image library
import matplotlib.pyplot as plt               # making plots
%matplotlib inline
from IPython.display import display           # displaying ?

import warnings                               # ignoring unnecessary python warnings
warnings.filterwarnings('ignore')

from keras.applications import VGG16                         # pretrained CNN
from keras.callbacks import ModelCheckpoint                  # furter train the saved model
from keras import models, layers, optimizers                 # building DNN is keras
from tensorflow.keras.models import load_model               # load saved model
from keras.preprocessing.image import ImageDataGenerator     # preparing image data for training

In [ ]:
conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 input_shape=(150, 150, 3))

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

conv_base.trainable = False

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

58889256/58889256 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16,812,353
Trainable params: 2,097,665
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
base_dir = '/content/drive/MyDrive'

train_dir = os.path.join(base_dir, 'Forest Fire Dataset/Training')
train_dir_fire = os.path.join(train_dir, 'fire')
train_dir_nofire = os.path.join(train_dir, 'nofire')

test_dir = os.path.join(base_dir, 'Forest Fire Dataset/Testing')
test_dir_fire = os.path.join(test_dir, 'fire')
test_dir_nofire = os.path.join(test_dir, 'nofire')

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='binary')

Found 1520 images belonging to 2 classes.
Found 369 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator, epochs=30,
                    validation_data=test_generator)

Epoch 1/30
48/48 [==============================] - 576s 12s/step - loss: 0.4512 - acc: 0.7934 - val_loss: 0.3015 - val_acc: 0.8808
Epoch 2/30
48/48 [==============================] - 555s 12s/step - loss: 0.3121 - acc: 0.8704 - val_loss: 0.2377 - val_acc: 0.9268
Epoch 3/30
48/48 [==============================] - 587s 12s/step - loss: 0.2650 - acc: 0.8921 - val_loss: 0.2034 - val_acc: 0.9350
Epoch 4/30
48/48 [==============================] - 582s 12s/step - loss: 0.2302 - acc: 0.8980 - val_loss: 0.1720 - val_acc: 0.9485
Epoch 5/30
48/48 [==============================] - 549s 11s/step - loss: 0.2102 - acc: 0.9197 - val_loss: 0.1662 - val_acc: 0.9404
Epoch 6/30
48/48 [==============================] - 580s 12s/step - loss: 0.1987 - acc: 0.9224 - val_loss: 0.1927 - val_acc: 0.9268


In [ ]:
model.save('/content/drive/MyDrive/Untitled folder/VGG16_lr-4.h5')

In [ ]:
# Dictionary to extract the numbers
hist_dict = history.history

# Training and validation accuracy
training_acc = hist_dict['acc']
validation_acc = hist_dict['val_acc']

# Training and validation loss
training_loss = hist_dict['loss']
validation_loss = hist_dict['val_loss']

# Number of epochs
epoches = range(1, 1 + len(training_acc))

In [ ]:
def plot_func(entity):

    '''
    This function produces plot to compare the performance
    between train set and validation set.
    entity can be loss of accuracy.
    '''

    plt.figure(figsize=(8, 5))
    plt.plot(epoches, eval('training_' + entity), 'r')
    plt.plot(epoches, eval('validation_' + entity), 'b')
    plt.legend(['Training ' + entity, 'Validation ' + entity])
    plt.xlabel('Epoches')
    plt.ylabel(entity)
    plt.show()

In [ ]:
plot_func('loss')

In [ ]:
plot_func('acc')

In [ ]:
conv_base.trainable = True
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    else:
        set_trainable = False

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-5),
             metrics=['acc'])

In [ ]:
history = model.fit_generator(train_generator, epochs=30,
                             validation_data=test_generator)

Epoch 1/30
48/48 [==============================] - 1691s 35s/step - loss: 0.1838 - acc: 0.9349 - val_loss: 0.0937 - val_acc: 0.9648
Epoch 2/30
48/48 [==============================] - 1687s 35s/step - loss: 0.0997 - acc: 0.9651 - val_loss: 0.0942 - val_acc: 0.9702
Epoch 3/30
48/48 [==============================] - 1674s 35s/step - loss: 0.0804 - acc: 0.9743 - val_loss: 0.1601 - val_acc: 0.9512
Epoch 4/30
48/48 [==============================] - 1685s 35s/step - loss: 0.0742 - acc: 0.9737 - val_loss: 0.0651 - val_acc: 0.9756
Epoch 5/30
48/48 [==============================] - 1681s 35s/step - loss: 0.0625 - acc: 0.9776 - val_loss: 0.0594 - val_acc: 0.9756
Epoch 6/30
48/48 [==============================] - 1674s 35s/step - loss: 0.0306 - acc: 0.9875 - val_loss: 0.0682 - val_acc: 0.9783
Epoch 7/30
18/48 [==========>...................] - ETA: 15:39 - loss: 0.0374 - acc: 0.9839

In [ ]:
model.save('/content/drive/MyDrive/Untitled folder/VGG16_fine_tuned.h5')

In [ ]:
hist_dict = history.history

training_accuracy = hist_dict['acc']
validation_accuracy = hist_dict['val_acc']

training_loss = hist_dict['loss']
validation_loss = hist_dict['val_loss']

epoches = range(1, 1 + len(training_acc))

In [ ]:
plot_func('loss')

In [ ]:
plot_func('accuracy')

In [ ]:
# Loading the saved model
model = load_model('/content/drive/MyDrive/Untitled folder/VGG16_fine_tuned.h5')

In [ ]:
# taking first batch from the generator
img, label = test_generator[0]

# Predicting the images from the first batch
pred = np.round(model.predict(img)).flatten()

In [ ]:
len(img)

In [ ]:
# Numeric to semantic labels
label_dict = {1.0: 'No fire', 0.0: 'Fire'}

# Generating collage of plots
fig = plt.figure(figsize=(10, 9))
plt.title('Classification by the model')
plt.axis('off')

for i, img_i in enumerate(img[:20]):
    ax = fig.add_subplot(4, 5, i+1)
    plt.axis('off')
    plt.title(label_dict[pred[i]], y=-0.2)
    ax.imshow(img_i)

In [ ]:
# Lists for missed fire images and missed non-fire images
msd_fire = []
msd_nofire = []

# Iterating through all the batches
for j in range(31):
    img, label = test_generator[j]
    pred = np.round(model.predict(img)).flatten()
    bool_list = label == pred

    # bool_list is False when there is misclassification
    for i, e in enumerate(bool_list):
        if e == False:

            # separating labels (fire and non-fire)
            if label[i] == 0:
                msd_fire.append(img[i])
            else:
                msd_nofire.append(img[i])

In [ ]:
# total number of sample in train set in each class
n_class = 1001

# number of misclassified fire and non-fire images
nm_fire, nm_nofire = len(msd_fire), len(msd_nofire)

# confusion matrix (flattened)
conf_mat = [n_class-nm_fire, nm_fire, nm_nofire, n_class-nm_nofire]

# Plot the confusion matrix as an image
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()
# visualization of confusion matrix
fig = plt.figure(figsize=(4, 4))
plt.title("Confusion Matrix")
plt.ylabel('True labels')
class_names=["fire","nonfire"]
plt.xlabel('Predicted labels')
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=90)
plt.yticks(tick_marks, class_names)
plt.axis()
for i, j in enumerate(conf_mat):
    ax = fig.add_subplot(2, 2, i+1)
    ax.imshow([[j]], vmin=0, vmax=1000, cmap='copper_r')
    ax.text(-0.2, 0.1, j, c='r', fontsize=30)
    ax.axis('off')





# bringing blocks tighter
fig.tight_layout()
fig.show()

In [ ]:
fig = plt.figure(figsize=(10, 7))
plt.title('Fire images classified as non-fire')
plt.axis('off')
for i, img_i in enumerate(msd_fire):
    ax = fig.add_subplot(4, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

In [ ]:
fig = plt.figure(figsize=(10, 4))
plt.title('Non-fire images classified as fire')
plt.axis('off')
for i, img_i in enumerate(msd_nofire):
    ax = fig.add_subplot(2, 5, i+1)
    ax.imshow(img_i)
    ax.axis('off')

In [ ]:
checkpoint = ModelCheckpoint('VGG16_fine_tuned.h5',
                             monitor='loss', verbose=1,
                             save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
hst_cntd = model.fit(train_generator, epochs=5,
                     steps_per_epoch = 15,
                     validation_data=test_generator,
                     validation_steps = 30,
                     callbacks=callbacks_list)

In [ ]:
import pandas as pd
import os

# Create an empty dataframe
data = pd.DataFrame(columns=['image_path', 'label'])

# Define the labels/classes
labels = {'/content/drive/MyDrive/Forest Fire Dataset/Training/fire' : 'Fire',
          '/content/drive/MyDrive/Forest Fire Dataset/Training/nofire' : 'No Fire',
           }

In [ ]:
# Loop over the train, test, and val folders and extract the image path and label
for folder in labels:
    for image_name in os.listdir(folder):
        image_path = os.path.join(folder, image_name)
        label = labels[folder]
        data = data.append({'image_path': image_path, 'label': label}, ignore_index=True)

In [ ]:
data.to_csv('image_dataset.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [ ]:
# Load the dataset
df = pd.read_csv("/content/image_dataset.csv")

In [ ]:
df

In [ ]:
# Split the dataset into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Pre-process the data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   rotation_range=45,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(dataframe=train_df,
                                                    x_col="image_path",
                                                    y_col="label",
                                                    target_size=(255, 255),
                                                    batch_size=32,
                                                    class_mode="categorical")

test_generator = test_datagen.flow_from_dataframe(dataframe=test_df,
                                                  x_col="image_path",
                                                  y_col="label",
                                                  target_size=(255, 255),
                                                  batch_size=32,
                                                  class_mode="categorical")

In [ ]:
# Build a deep learning model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(255, 255, 3), activation='relu'))
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=30,validation_data=test_generator)

In [ ]:
num_samples = test_df.shape[0]
num_samples

In [ ]:
score = model.evaluate(test_generator,
                       steps=num_samples//32+1,
                       )

In [ ]:
import matplotlib.pyplot as plt

# Plot the loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# Plot the accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:

from tensorflow.keras.models import load_model

model.save('Model.h5')
# Load the model
model = load_model("Model.h5")

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define the class names
class_names = ['Fire','No Fire']

In [ ]:
# Load an image from the test set
img = load_img("/content/drive/MyDrive/Forest Fire Dataset/Testing/fire_0015.jpg", target_size=(255, 255))

In [ ]:
# Convert the image to an array
img_array = img_to_array(img)
img_array

In [ ]:
img_array.shape

In [ ]:
# Normalize the image pixel values to the range [0, 1]
img_array = img_array / 255.0
img_array

In [ ]:
import numpy as np
img_array = np.reshape(img_array, (1, 255, 255, 3))

In [ ]:
# Get the model predictions
predictions = model.predict(img_array)

In [ ]:
# Get the class index with the highest predicted probability
class_index = np.argmax(predictions[0])

# Get the predicted class label
predicted_label = class_names[class_index]

print("The image is predicted to be '{}'.".format(predicted_label))

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [ ]:
predictions = model.predict_generator(test_generator)

In [ ]:
predictions

In [ ]:
actual_labels = test_generator.classes
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels



In [ ]:
cm = confusion_matrix(actual_labels, predicted_labels)

In [ ]:
print(cm)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import itertools

# Confusion matrix


# Plot the confusion matrix as an image
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.colorbar()

# Add the class labels to the plot
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)

# Add values to the plot
threshold = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j],
             horizontalalignment="center",
             color="white" if cm[i, j] > threshold else "black")

plt.tight_layout()
plt.ylabel("True label")
plt.xlabel("Predicted label")
plt.show()